In [ ]:
from pylab import *
import os
import sys
from sklearn.mixture import GaussianMixture
import pickle
from PIL import Image

In [ ]:
dataset_dir = 'rgbd_dataset_freiburg3_long_office_household'
depth_dir = 'depth'
gmm_dir = 'gmm_qqvga'

# og 
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y

# fri3 
fx = 535.4  # focal length x
fy = 539.2  # focal length y
cx = 320.1  # optical center x
cy = 247.6  # optical center y
factor = 5000 # for the 16-bit PNG files
# OR: factor = 1 # for the 32-bit float images in the ROS bag files

full_depth_dir = os.path.join(dataset_dir,depth_dir)


In [ ]:
y_grid = np.repeat(np.arange(480)[:,None],640,1)
x_grid = np.repeat(np.arange(640)[None,:],480,0)

def img_to_pts(img):
    Z = img/factor
    X = (x_grid-cx) * Z /fx
    Y = (y_grid-cy) * Z /fy
    #for v in range(depth_image.height):
    #  for u in range(depth_image.width):
    #    Z = depth_image[v,u] / factor;
    #    X = (u - cx) * Z / fx;
    #    Y = (v - cy) * Z / fy;
    X = X[::4,::4]
    Y = Y[::4,::4]
    Z = Z[::4,::4]

    xl = X[Z > 0]
    yl = Y[Z > 0]
    zl = Z[Z > 0]
    n = zl.shape[0]
    idx = np.random.randint(n, size=n//16)

    return np.vstack([xl,yl,zl]).T

def load_data(file):
    image = Image.open(file)   
    pixel = np.array(image)
    return img_to_pts(pixel)

dataset = []
names = []
for fl in sorted(os.listdir(full_depth_dir)):
    name,ext = os.path.splitext(fl)
    data = load_data(os.path.join(full_depth_dir,fl))
    dataset.append(data)
    names.append(name)


In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource

In [ ]:
name_times = [float(_) for _ in names]

In [ ]:
import pandas as pd
gt = pd.read_csv('rgbd_dataset_freiburg3_long_office_household-groundtruth.txt',' ',comment='#',header=None,names='timestamp tx ty tz qx qy qz qw'.split(' '))
def row_to_vec(row):
    a = np.array(row)
    return a[1:4], a[4:]
from scipy.spatial.distance import cdist, pdist
time_dists = cdist(np.array(gt.timestamp)[:,None],np.array(name_times)[:,None])
matches = time_dists.argmin(axis=0)

start_idx = matches[25]
end_idx = matches[125]

frm = row_to_vec(gt.iloc[start_idx])
to = row_to_vec(gt.iloc[end_idx])
print(gt.timestamp[end_idx] - gt.timestamp[start_idx])

start_idx,end_idx


In [ ]:
for i in range(len(matches)-1):
    f1idx = matches[i]
    f2idx = matches[i+1]
    f1 = row_to_vec(gt.iloc[f1idx])
    f2 = row_to_vec(gt.iloc[f2idx])

    print(f1)

In [ ]:
frm,to

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)

start_frame = dataset[start_idx][np.random.randint(0,dataset[start_idx].shape[0],1000)]
end_frame = dataset[end_idx][np.random.randint(0,dataset[end_idx].shape[0],1000)]

ax.scatter(start_frame[:,0],start_frame[:,1],start_frame[:,2])
ax.scatter(end_frame[:,0],end_frame[:,1],end_frame[:,2])
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1.5,1.5)
ax.set_zlim(0,3)

In [ ]:
import transforms3d
fig = plt.figure()
ax = Axes3D(fig)

start_frame = dataset[start_idx][np.random.randint(0,dataset[start_idx].shape[0],1000)]
end_frame = dataset[end_idx][np.random.randint(0,dataset[end_idx].shape[0],1000)]

r1 = transforms3d.quaternions.quat2mat(to[1])
r2 = transforms3d.quaternions.quat2mat(frm[1])
r1 = np.identity(3)
r2 = np.identity(3)

end_frame = end_frame + (to[0]-frm[0])
end_frame = (r1 @ r2.T @(end_frame - end_frame.mean(0)).T).T + end_frame.mean(0)

ax.scatter(start_frame[:,0],start_frame[:,1],start_frame[:,2])
ax.scatter(end_frame[:,0],end_frame[:,1],end_frame[:,2])
ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_zlim(0.5,4.5)